In [47]:
import tensorflow as tf
import tensorflow_datasets as tfds
import tensorflow_hub as hub


In [48]:
tf.__version__

'2.5.0'

In [49]:
tfds.list_builders()

['abstract_reasoning',
 'accentdb',
 'aeslc',
 'aflw2k3d',
 'ag_news_subset',
 'ai2_arc',
 'ai2_arc_with_ir',
 'amazon_us_reviews',
 'anli',
 'arc',
 'bair_robot_pushing_small',
 'bccd',
 'beans',
 'big_patent',
 'bigearthnet',
 'billsum',
 'binarized_mnist',
 'binary_alpha_digits',
 'blimp',
 'bool_q',
 'c4',
 'caltech101',
 'caltech_birds2010',
 'caltech_birds2011',
 'cars196',
 'cassava',
 'cats_vs_dogs',
 'celeb_a',
 'celeb_a_hq',
 'cfq',
 'chexpert',
 'cifar10',
 'cifar100',
 'cifar10_1',
 'cifar10_corrupted',
 'citrus_leaves',
 'cityscapes',
 'civil_comments',
 'clevr',
 'clic',
 'clinc_oos',
 'cmaterdb',
 'cnn_dailymail',
 'coco',
 'coco_captions',
 'coil100',
 'colorectal_histology',
 'colorectal_histology_large',
 'common_voice',
 'coqa',
 'cos_e',
 'cosmos_qa',
 'covid19sum',
 'crema_d',
 'curated_breast_imaging_ddsm',
 'cycle_gan',
 'deep_weeds',
 'definite_pronoun_resolution',
 'dementiabank',
 'diabetic_retinopathy_detection',
 'div2k',
 'dmlab',
 'downsampled_imagenet',
 

In [50]:
train_data   , test_data = tfds.load(name='imdb_reviews',split=(tfds.Split.TRAIN,tfds.Split.TEST),as_supervised=True)

In [51]:
type(train_data)

tensorflow.python.data.ops.dataset_ops.PrefetchDataset

In [52]:
train_examples_batches , train_labels_batches = next(iter(train_data.batch(10)))
print(train_examples_batches)

tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell asleep because the film was rubbish. The plot de

In [53]:
print(train_labels_batches)

tf.Tensor([0 0 0 1 1 1 0 0 0 0], shape=(10,), dtype=int64)


In [54]:
#do word_to vec conversion for a number
pre_trained_model = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim-with-oov/1"

In [55]:
hub_layer = hub.KerasLayer(pre_trained_model,input_shape=[] , dtype = tf.string,trainable=True)

In [56]:
hub_layer(train_examples_batches[:2])

<tf.Tensor: shape=(2, 20), dtype=float32, numpy=
array([[ 1.668895  , -3.918618  ,  3.7993348 , -1.4097565 , -3.3214598 ,
        -0.9996616 , -2.0233727 ,  1.2953283 ,  5.0233593 , -1.0990871 ,
        -2.0879953 , -1.2672297 , -0.18820252, -0.07738297, -2.9172704 ,
         1.7943126 ,  1.2375495 , -1.5559573 , -2.4811747 , -0.7813505 ],
       [ 1.7287169 , -2.4736629 ,  3.3280423 ,  0.9174705 , -4.062213  ,
        -4.791508  , -2.7125058 ,  1.2155541 ,  3.7063696 , -0.95357007,
        -1.855471  ,  1.2775491 , -0.07673654,  0.6835056 , -4.822123  ,
         2.6815715 ,  4.479097  , -2.360208  , -3.2802453 , -1.5463475 ]],
      dtype=float32)>

In [57]:
#actual neural network
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16,activation="relu"))
model.add(tf.keras.layers.Dense(1,activation="sigmoid"))
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer_1 (KerasLayer)   (None, 20)                389380    
_________________________________________________________________
dense_4 (Dense)              (None, 16)                336       
_________________________________________________________________
dense_5 (Dense)              (None, 1)                 17        
Total params: 389,733
Trainable params: 389,733
Non-trainable params: 0
_________________________________________________________________


In [58]:
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

In [59]:
model.fit(train_data.shuffle(10000).batch(512),epochs=20,verbose=1)

Epoch 1/20
49/49 [==============================] - 4s 61ms/step - loss: 0.7010 - accuracy: 0.5870
Epoch 2/20
49/49 [==============================] - 4s 62ms/step - loss: 0.5859 - accuracy: 0.6998
Epoch 3/20
49/49 [==============================] - 4s 60ms/step - loss: 0.5357 - accuracy: 0.7431
Epoch 4/20
49/49 [==============================] - 4s 63ms/step - loss: 0.4867 - accuracy: 0.7806
Epoch 5/20
49/49 [==============================] - 4s 63ms/step - loss: 0.4367 - accuracy: 0.8121
Epoch 6/20
49/49 [==============================] - 4s 61ms/step - loss: 0.3904 - accuracy: 0.8391
Epoch 7/20
49/49 [==============================] - 4s 62ms/step - loss: 0.3499 - accuracy: 0.8589
Epoch 8/20
49/49 [==============================] - 4s 61ms/step - loss: 0.3162 - accuracy: 0.8754
Epoch 9/20
49/49 [==============================] - 4s 61ms/step - loss: 0.2882 - accuracy: 0.8880
Epoch 10/20
49/49 [==============================] - 4s 61ms/step - loss: 0.2646 - accuracy: 0.8980
Epoch 11/

In [60]:
model.predict(["this is the worst movie that i have ever seen"])

array([[0.04122329]], dtype=float32)

In [68]:
import nltk
nltk.download('punkt')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('stopwords')
stop_word=set(stopwords.words('english'))
sentence  = "this movie is a good movie"
word_tokens = word_tokenize(sentence)
updated_sentence = [w for w in word_tokens if not w.lower() in stop_word]
updated_sentence = []
 
for w in word_tokens:
    if w not in stop_word:
        updated_sentence.append(w)
print(updated_sentence)
model.predict(updated_sentence)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
['movie', 'good', 'movie']


array([[0.43559578],
       [0.88375413],
       [0.43559578]], dtype=float32)